<a href="https://colab.research.google.com/github/AndresPlazas19931504/PROYECTOINTEGRADOR-PREICA2501B020071/blob/main/S25_Evidencia_de_Aprendizaje_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
from IPython.display import display
import io

file_path = "2019-Oct.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mkechinov/ecommerce-behavior-data-from-multi-category-store",
  file_path,
)

<ipython-input-1-b979563c9fc9>:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 1.61G/1.61G [00:18<00:00, 95.6MB/s]

Extracting zip of 2019-Oct.csv...


In [ ]:
display(df.head())

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['event_time'].unique()
df['event_time'].value_counts()

In [ ]:
df['event_type'].unique()
df['event_type'].value_counts()

In [ ]:
df['product_id'].unique()
df['product_id'].value_counts()

In [ ]:
df['category_id'].unique()
df['category_id'].value_counts()

In [ ]:
df['category_code'].unique()
df['category_code'].value_counts()

In [ ]:
df['brand'].unique()
df['brand'].value_counts()

In [ ]:
df['price'].unique()
df['price'].value_counts()

In [ ]:
df['user_id'].unique()
df['user_id'].value_counts()

In [ ]:
df['user_session'].unique()
df['user_session'].value_counts()

In [ ]:
!pip install kagglehub[pandas-datasets]
!pip install --upgrade ydata-profiling
!pip install --no-cache-dir ydata-profiling


import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import ydata_profiling as pp
from ydata_profiling import ProfileReport

file_path = "2019-Oct.csv"
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mkechinov/ecommerce-behavior-data-from-multi-category-store",
  file_path,
)

df_muestra = df.sample(frac=0.125)

reporte = ProfileReport(df_muestra, minimal=False)
reporte.to_file(output_file='reporte.html')